# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [3]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [78]:
# your code here
orders = pd.read_csv('Orders.zip')
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [79]:
#Version 1
# your code here
#sub problem 1
customer_spending=orders.groupby(by = ["CustomerID"])["amount_spent"].sum()

#sub problem 2
position=list(customer_spending.quantile([.75, .95]))

#Middle step
customer_spending=customer_spending.rename("Aggregate spending")
orders_df=orders.merge(customer_spending, on="CustomerID")

#sub problem 3
#orders["customer type"]=orders["Aggregate spending"].apply(lambda x: "VIP" if x>position[1] else "Preferred" if position[0]<x<position[1] else "Normal")
orders_df["VIP"]=orders_df["Aggregate spending"].apply(lambda x: "VIP" if x>position[1] else None)
orders_df["Preferred"]=orders_df["Aggregate spending"].apply(lambda x: "Preferred" if position[0]<x<position[1] else None)

In [39]:
# Version 2_ much simpler _after ter iluminação divina
main_df=orders.groupby(by = ["CustomerID","Country"])["amount_spent"].sum()

#customer_spending["ID"] = customer_spending.index
main_df = pd.DataFrame(main_df)
main_df = main_df.reset_index()
main_df['Tier'] = pd.qcut(main_df['amount_spent'], q = [0,0.75,0.95,1], labels = ['Normal',' Preferred', 'VIP'])

main_df.head()

,CustomerID,Country,amount_spent,Tier
0,12346,United Kingdom,77183.60,VIP
1,12347,Iceland,4310.00,Preferred
2,12348,Finland,1797.24,Preferred
3,12349,Italy,1757.55,Preferred
4,12350,Norway,334.40,Normal


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [80]:
#....Version 1 (old version)
# your code here
#location_type=orders.groupby(by = ["Country","customer type"])["CustomerID"].nunique()
location_type=orders_df.groupby(by = ["Country","VIP"])["CustomerID"].nunique()
location_type[location_type==location_type.max()]

Country         VIP
United Kingdom  VIP    177
Name: CustomerID, dtype: int64

In [101]:
#....Version 2
vip_group=pd.DataFrame(main_df.groupby(by = ["Country","Tier"])["CustomerID"].nunique())  #count would also work
vip_group=vip_group.reset_index()
vip_group[vip_group["CustomerID"] == (vip_group[vip_group["Tier"]=="VIP"]["CustomerID"].max())]

,Country,Tier,CustomerID
107,United Kingdom,VIP,178


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [115]:
# ....Version 1 (old version)
location_type=orders_df.groupby(by = ["Country","CustomerID"])["Preferred","VIP"].nunique()
location_type=location_type.groupby(by = ["Country"])["Preferred","VIP"].sum()
location_type["Top_clients"]=location_type["Preferred"] + location_type["VIP"]
location_type[location_type["Top_clients"]==location_type["Top_clients"].max()]

C:\Users\maria\Anaconda3\envs\Ironhack\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  
C:\Users\maria\Anaconda3\envs\Ironhack\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,Preferred,VIP,Top_clients
Country,,,
United Kingdom,755,177,932


In [116]:
#....Version 2
vip_group=pd.DataFrame(main_df.groupby(by = ["Country","Tier"])["CustomerID"].nunique())  #count would also work
vip_group=vip_group.reset_index()
vip_preferred = pd.DataFrame(vip_group[vip_group["Tier"]!="Normal"].groupby(by="Country")["CustomerID"].sum())
vip_preferred[vip_preferred["CustomerID"]==vip_preferred["CustomerID"].max()]

,CustomerID
Country,
United Kingdom,934
